In [8]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import scipy
from sklearn import preprocessing

In [9]:
fmli = pd.read_csv("fmli_15_clean.csv")
fmli.drop(["DETTACHED"], axis = 1, inplace = True)

In [10]:
sf2010 = pd.read_csv("fmli_2010_clean.csv")

In [11]:
dm = sf2010[["OWN", "DEGREE", "HHSIZE", "INCOME", "ROOMS", "VEHICLES"]]
dm.replace(".", np.nan, inplace = True)
dm["ROOMS"] = pd.to_numeric(dm["ROOMS"])

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##### Normalize data frame

In [12]:
# Log every variables, save it in a new data frame for future use.
log = np.log(fmli)
log["OWN"] = fmli["OWN"]
log["DEGREE"] = fmli["DEGREE"]
log["VEHICLES"] = fmli["VEHICLES"]

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


In [13]:
# Log every variables, save it in a new data frame for future use.
log_dm = np.log(dm)
log_dm["OWN"] = dm["OWN"]
log_dm["DEGREE"] = dm["DEGREE"]
log_dm["VEHICLES"] = dm["VEHICLES"]
log_dm = log_dm[log_dm.columns[[2, 3, 4, 0, 1, 5]]]
log_dm["CUID"] = sf2010["CUID"]
log_dm["NEWID"] = sf2010["NEWID"]

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


##### Run models and Apply

In [14]:
pred_y = pd.DataFrame()

for i in range(16, len(log.columns)):
    df = log[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES", log.columns[i]]]
    df.replace(np.Inf, np.nan, inplace = True)
    df.replace(-np.Inf, np.nan, inplace = True)
    df.dropna(inplace = True)
    df.reset_index(inplace = True)
    X = df[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES"]]
    Y = df[log.columns[i]]
    X1 = sm.add_constant(X)
    model = linear_model.Lasso(alpha=0.03).fit(X1, Y)
    log_dm.replace(np.Inf, np.nan, inplace = True)
    log_dm.replace(-np.Inf, np.nan, inplace = True)
    log_dm.dropna(inplace = True)
    Xp = log_dm[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES"]]
    log_dm1 = sm.add_constant(Xp)
    pred_y[log.columns[i]] = np.array(model.predict(log_dm1))

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the do

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is depr

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is depr

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is depr

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is depr

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is depr

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is depr

In [15]:
result = pred_y
for i in pred_y.columns:
    result[i] = np.exp(pred_y[i])

In [16]:
result["CUID"] = log_dm["CUID"]
result["NEWID"] = log_dm["NEWID"]

In [17]:
cols = list(result.columns)
cols = [cols[len(cols) - 2]] + [cols[len(cols) - 1]] + cols[:len(cols) - 2]
result = result[cols]

In [18]:
result.head()

,CUID,NEWID,TotalExpenditures,TotalFood,FoodAtHome,FoodAwayFromHome,AlcoholicBeverages,Housing,Shelter,OtherLodging,...,PetsToysAndPlaygroundEquipment,OtherEntertainment,PersonalCare,Reading,Education,TobaccoAndSmokingSupplies,MiscellaneousExpenditures,CashContributions,PersonalInsuranceAndPensions,AdjustedTotalExpenditures
0,199554.0,1995545.0,22238.325816,3631.410398,2537.847016,1144.560083,271.280373,7473.744118,3040.006654,1178.605301,...,420.345305,382.740084,191.435421,191.671677,1345.815597,743.599101,247.891071,876.286083,2237.769675,22238.325816
1,199556.0,1995565.0,39615.808113,4502.100653,2733.163436,1518.426278,353.949922,9855.317430,4720.775028,1496.205118,...,534.979511,565.754226,236.481145,212.254481,2764.475522,837.499756,299.996245,1584.278906,4227.060337,39615.808113
2,199558.0,1995585.0,46650.167776,5098.749922,2935.946745,1802.160297,417.557128,11705.485003,9633.986594,1752.778202,...,561.690694,626.698565,261.036896,222.165574,2839.305302,877.072030,329.937881,1389.331247,6993.389466,46650.167776
3,199559.0,1995595.0,29109.792936,4673.696102,3248.802695,1364.254098,302.737058,9061.102600,3691.832060,1298.038419,...,455.426533,454.187629,218.962791,200.170113,1391.523506,800.845437,273.633972,980.524179,3516.504099,29109.792936
4,199560.0,1995605.0,20456.213273,3409.610997,2363.993628,1098.234381,257.498763,7531.315204,5646.867662,1190.337276,...,356.493151,322.593477,171.753966,168.260806,1335.798775,710.326829,206.023961,625.867486,1436.791312,20456.213273


In [19]:
result.to_csv("pred_fmli_2010.csv")

##### Same procedure for fmli2000

In [35]:
sf2000 = pd.read_csv("fmli_2000_clean.csv")

In [36]:
dm = sf2000[["OWN", "DEGREE", "HHSIZE", "INCOME", "ROOMS", "VEHICLES"]]
dm.replace(".", np.nan, inplace = True)
dm["ROOMS"] = pd.to_numeric(dm["ROOMS"])

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
# Log every variables, save it in a new data frame for future use.
log_dm = np.log(dm)
log_dm["OWN"] = dm["OWN"]
log_dm["DEGREE"] = dm["DEGREE"]
log_dm["VEHICLES"] = dm["VEHICLES"]
log_dm = log_dm[log_dm.columns[[2, 3, 4, 0, 1, 5]]]
log_dm["NEWID"] = sf2000["NEWID"]

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


In [38]:
pred_y = pd.DataFrame()

for i in range(16, len(log.columns)):
    df = log[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES", log.columns[i]]]
    df.replace(np.Inf, np.nan, inplace = True)
    df.replace(-np.Inf, np.nan, inplace = True)
    df.dropna(inplace = True)
    df.reset_index(inplace = True)
    X = df[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES"]]
    Y = df[log.columns[i]]
    X1 = sm.add_constant(X)
    model = linear_model.Lasso(alpha=0.03).fit(X1, Y)
    log_dm.replace(np.Inf, np.nan, inplace = True)
    log_dm.replace(-np.Inf, np.nan, inplace = True)
    log_dm.dropna(inplace = True)
    Xp = log_dm[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES"]]
    log_dm1 = sm.add_constant(Xp)
    pred_y[log.columns[i]] = np.array(model.predict(log_dm1))

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the do

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the do

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the do

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the do

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the do

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the do

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the do

In [39]:
result = pred_y
for i in pred_y.columns:
    result[i] = np.exp(pred_y[i])

In [40]:
result["NEWID"] = log_dm["NEWID"]

In [41]:
cols = list(result.columns)
cols = [cols[len(cols) - 1]] + cols[:len(cols) - 1]
result = result[cols]

In [42]:
result.head()

,NEWID,TotalExpenditures,TotalFood,FoodAtHome,FoodAwayFromHome,AlcoholicBeverages,Housing,Shelter,OtherLodging,UtilitiesFuelsAndPublicServices,...,PetsToysAndPlaygroundEquipment,OtherEntertainment,PersonalCare,Reading,Education,TobaccoAndSmokingSupplies,MiscellaneousExpenditures,CashContributions,PersonalInsuranceAndPensions,AdjustedTotalExpenditures
0,977915.0,10792.772592,3002.459494,2908.805484,631.318648,132.966634,4405.700550,2311.564571,614.680057,396.965485,...,240.691082,217.684455,117.959045,130.599301,434.359302,634.941604,138.632409,345.138555,231.846303,10792.772592
1,977925.0,25863.024849,4395.243826,3140.887760,1258.284176,280.019544,8325.231398,3324.522531,1202.118167,567.921122,...,418.985759,420.889797,200.872854,188.826567,1249.800089,783.582756,252.286799,850.175277,2670.699999,25863.024849
2,977935.0,19691.362832,3496.240746,2492.561948,1104.850534,261.705815,7368.644201,4919.940578,1169.669160,369.667767,...,388.266942,338.111376,184.585972,185.671016,1089.888900,717.799691,233.692446,647.898812,1926.872542,19691.362832
3,977975.0,24997.273614,3135.928881,1874.320626,1265.202170,327.917914,8203.590288,4132.120452,1460.848507,474.024943,...,455.278948,396.936275,197.219348,197.745113,2455.305727,736.619704,257.320813,1079.771025,2924.483011,24997.273614
4,977995.0,36801.141080,5087.035302,3277.918819,1584.877748,345.569759,11192.245660,5375.720909,1574.959052,702.520189,...,501.486209,470.390153,249.945214,208.754306,2036.453428,805.467973,285.068919,1286.292745,4614.506631,36801.141080


In [43]:
result.to_csv("pred_fmli_2000.csv")